In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
plt.style.use('ggplot')

%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys

In [3]:
def get_data_year(path, year):
    file_name = [file for file in os.listdir(path) if file[0:4] == str(year)][0]
    df = pd.read_csv(path + file_name)
    return df


data_path ='../data/'
df = get_data_year(data_path, 2023)

Obtaining 2023 data for LEC, LPL. LCK participants at MSI. 

In [4]:
lec_teams_msi = ['G2 Esports', 'MAD Lions']


In [11]:
def flatten_cols(df):
    df.columns = ['_'.join(x) for x in df.columns.to_flat_index()]
    return df
# .pipe(flatten_cols)#.to_flat_index()#.reset_index()


df_league = df[df['league'] == 'LEC']
league_teams = df_league['teamname'].unique()

split__winrate_summary = (
    df_league
    [df_league['position'] == 'team'] # filter
    .groupby(['split', 'playoffs', 'teamname']) # by
    .agg(
        games_won = pd.NamedAgg(column='result', aggfunc='sum'),
        games_played = pd.NamedAgg(column='result', aggfunc='count'),
        # winrate = pd.NamedAgg(column='result', aggfunc=lambda x: x.count()/x.sum()*100),
    )
    .sort_values(by=['split','playoffs', 'games_won'], ascending=[False, True, False])
    .reset_index()
    .assign(winrate=lambda x: round(x['games_won']/x['games_played']*100, 2))
)
split__winrate_summary

,split,playoffs,teamname,games_won,games_played,winrate
0,Winter,0,MAD Lions,11,16,68.75
1,Winter,0,G2 Esports,10,14,71.43
2,Winter,0,SK Gaming,10,15,66.67
3,Winter,0,Team Vitality,9,16,56.25
4,Winter,0,KOI,8,13,61.54
5,Winter,0,Team BDS,7,15,46.67
6,Winter,0,Astralis,6,17,35.29
7,Winter,0,Team Heretics,5,14,35.71
8,Winter,0,Fnatic,2,9,22.22
9,Winter,0,Excel Esports,1,9,11.11


In [122]:
#msi_teams_main_regions = ['Bilibili Gaming','' ,'T1', 'Golden Guardians']

agg_column = lambda name: {
    f'{name}_mean': pd.NamedAgg(column=name, aggfunc='mean'), 
    f'{name}_std':pd.NamedAgg(column=name, aggfunc='std'),
    f'{name}_sum':pd.NamedAgg(column=name, aggfunc='sum'),
}

agg_column_pm = lambda name: {
    f'{name}_mean': pd.NamedAgg(column=name, aggfunc='mean'), 
    f'{name}_std':pd.NamedAgg(column=name, aggfunc='std'),
}

(
    df
    [(df['league'] == 'MSI')
    & (df['position'] != 'team')
    # & (df['teamname'].isin())
    ]
    .groupby(['position', 'playername', 'teamname'])
    .agg(
        **agg_column('kills'),
        **agg_column('deaths'),
        **agg_column('assists'),
        **agg_column_pm('dpm'),
        **agg_column_pm('cspm'),
    ).round(3)
    .sort_values(by=['position', 'kills_sum'], ascending=[True, False])
)

kills_mean  kills_std  kills_sum   
position playername  teamname                                               
bot      Stixxay     Golden Guardians         6.700      3.057         67  \
         Elk         Bilibili Gaming          7.750      2.915         62   
         Wako        PSG Talon                5.000      3.464         45   
         Hans Sama   G2 Esports               4.250      2.435         34   
         Gumayusi    T1                       9.333      0.577         28   
         Peyz        Gen.G                    6.000      5.033         24   
         Ceo         Movistar R7              3.143      2.734         22   
         Route       LOUD                     3.667      2.160         22   
         Sty1e       GAM Esports              4.400      3.782         22   
         Berserker   Cloud9                   5.000      2.646         15   
         Yutapon     DetonatioN FocusMe       3.500      3.873         14   
         Carzzy      MAD Lions                3.000      4.359          9   
jng      Yike        G2 Esports               5.250      3.536         42   
         XUN         Bilibili Gaming          4.250      2.866         34   
         JunJia      PSG Talon                3.444      2.698         31   
         River       Golden Guardians         2.600      1.265         26   
         Oddie       Movistar R7              2.571      1.813         18   
         Croc        LOUD                     2.833      0.753         17   
         Levi        GAM Esports              3.200      3.347         16   
         Peanut      Gen.G                    4.000      2.944         16   
         Oner        T1                       4.667      2.517         14   
         Blaber      Cloud9                   4.333      1.155         13   
         Elyoya      MAD Lions                4.333      4.041         13   
         Steal       DetonatioN FocusMe       2.750      0.957         11   
mid      Gori        Golden Guardians         4.500      2.461         45   
         ubao        PSG Talon                3.667      2.550         33   
         Yagao       Bilibili Gaming          3.875      1.727         31   
         Caps        G2 Esports               2.250      2.252         18   
         tinowns     LOUD                     2.667      2.805         16   
         Faker       T1                       4.667      0.577         14   
         Chovy       Gen.G                    3.250      1.893         13   
         Mireu       Movistar R7              1.714      2.059         12   
         Aria        DetonatioN FocusMe       2.750      2.500         11   
         Kati        GAM Esports              1.600      1.342          8   
         EMENES      Cloud9                   1.333      0.577          4   
         Nisqy       MAD Lions                1.000      0.000          3   
sup      Mikyx       G2 Esports               1.250      1.832         10   
         ON          Bilibili Gaming          1.250      1.389         10   
         Lyonz       Movistar R7              1.143      1.215          8   
         huhi        Golden Guardians         0.800      0.632          8   
         Hylissang   MAD Lions                1.333      0.577          4   
         Woody       PSG Talon                0.444      0.527          4   
         Delight     Gen.G                    0.750      0.957          3   
         Zin         GAM Esports              0.400      0.894          2   
         Zven        Cloud9                   0.667      1.155          2   
         Keria       T1                       0.333      0.577          1   
         Ceos        LOUD                     0.000      0.000          0   
         Harp        DetonatioN FocusMe       0.000      0.000          0   
top      BrokenBlade G2 Esports               4.750      4.234         38   
         Bin         Bilibili Gaming          3.375      2.134         27   
         Licorice    Golden Guardians

In [106]:
with pd.option_context('display.max_columns', None):
    # display(df.head(12))
    display(df_league.head(5))

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
2340,ESPORTSTMNT06_2767073,complete,NaN,LEC,2023,Winter,0,2023-01-21 17:27:32,1,13.01,1,Blue,top,Odoamne,oe:player:2b819506567c258f6fffdd4ec78ca0c,Excel Esports,oe:team:8f995652631df209ff11c4542abdcaf,K'Sante,Sylas,Ryze,Yuumi,Akali,Gwen,1826,0,5,5,2,11,18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3614,0.9529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,14965,491.7306,0.229778,989.9014,1580.7338,7,0.2300,3,0.0986,2,14,0.4600,10574,6573,215.9803,0.227650,10200,NaN,230.0,220.0,10,NaN,NaN,7.5575,3241.0,4806.0,84.0,3126.0,4960.0,69.0,115.0,-154.0,15.0,0.0,0.0,1.0,0.0,1.0,0.0,5066.0,8170.0,130.0,4931.0,7877.0,113.0,135.0,293.0,17.0,1.0,0.0,1.0,0.0,1.0,1.0
2341,ESPORTSTMNT06_2767073,complete,NaN,LEC,2023,Winter,0,2023-01-21 17:27:32,1,13.01,2,Blue,jng,Xerxe,oe:player:ab7f28ca18446cb18274a884b1525f7,Excel Esports,oe:team:8f995652631df209ff11c4542abdcaf,Wukong,Sylas,Ryze,Yuumi,Akali,Gwen,1826,0,0,3,5,11,18,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3614,0.9529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,5834,191.6977,0.089577,947.6451,685.5969,6,0.1972,16,0.5257,5,48,1.5772,8252,4251,139.6824,0.147234,8075,NaN,160.0,37.0,123,NaN,NaN,5.2574,2939.0,2950.0,58.0,3559.0,3539.0,62.0,-620.0,-589.0,-4.0,0.0,1.0,0.0,1.0,0.0,1.0,4406.0,4987.0,94.0,5783.0,5869.0,104.0,-1377.0,-882.0,-10.0,0.0,1.0,0.0,2.0,0.0,1.0
2342,ESPORTSTMNT06_2767073,complete,NaN,LEC,2023,Winter,0,2023-01-21 17:27:32,1,13.01,3,Blue,mid,Vetheo,oe:player:5eb466e63c91a4ccf328f21715e51ae,Excel Esports,oe:team:8f995652631df209ff11c4542abdcaf,Syndra,Sylas,Ryze,Yuumi,Akali,Gwen,1826,0,3,4,2,11,18,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3614,0.9529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,16135,530.1752,0.247743,613.6035,355.4326,15,0.4929,3,0.0986,7,26,0.8543,11858,7857,258.1709,0.272118,11375,NaN,289.0,273.0,16,NaN,NaN,9.4962,3080.0,4154.0,78.0,3266.0,4972.0,83.0,-186.0,-818.0,-5.0,0.0,1.0,1.0,0.0,1.0,0.0,4841.0,7127.0,135.0,5188.0,8123.0,145.0,-347.0,-996.0,-10.0,0.0,1.0,1.0,0.0,1.0,0.0
2343,ESPORTSTMNT06_2767073,complete,NaN,LEC,2023,Winter,0,2023-01-21 17:27:32,1,13.01,4,Blue,bot,Patrik,oe:player:1dad889ff236635dc1e03d02312e79c,Excel Esports,oe:team:8f995652631df209ff11c4542abdcaf,Varus,Sylas,Ryze,Yuumi,Akali,Gwen,1826,0,1,4,3,11,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3614,0.9529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,17872,587.2508,0.274413,431.1720,215.1588,11,0.3614,4,0.1314,2,21,0.6900,9853,5852,192.2892